# The Great Annual Fish Fry Migration, Part 2

![](https://media.giphy.com/media/wJt98DnjJHbJW46KsA/giphy.gif)

In this notebook, we load the updated fish fry GeoJSON to the database.

Currently the database in DynamoDB. In it, rows = GeoJSON features, and the columns = the top level GeoJSON properties (ID, properties, geometry, and type).

---

*NOTE* it is also entirely possible to do this with the Fish Fry API. For anyone with a fishfry.codeforpgh.com login but without direct access to the database, that's the way to do it. We'll cover that in another notebook.

---

In [13]:
import json
import decimal
import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
from tqdm import tqdm_notebook as tqdm

## helper functions

In [15]:
class DecimalEncoder(json.JSONEncoder):
    """use to convert any DynamoDB items stored as Decimal objects to numbers.
    """

    def default(self, o):
        if isinstance(o, decimal.Decimal):
            return float(o)
        return super(DecimalEncoder, self).default(o)

def decimal_encoder(json_obj):
    return json.loads(json.dumps(json_obj, cls=DecimalEncoder))

## Open the geojson

In [11]:
with open('updated_fishfrydb.geojson') as fp:
    fishfry_geojson = json.load(fp, parse_float=decimal.Decimal)

Note above how we convert any floating point values (e.g., coordinates) to Decimal objects when reading the GeoJSON file in here. DynamoDB needs floats to be submitted this way.

## Connect to the table

This assumes you have an AWS access key and AWS secret that gets you direct access to the database table.

In [10]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-2')
table = dynamodb.Table('FishFryDB')

In [9]:
print("Current number of entries:", table.item_count)

Current number of entries: 357


## Push the updates

The `put_item` method on the table will simply update items in place if the ID matches. Easy-peazy.

In [16]:
for fishfry in tqdm(fishfry_geojson['features']):
    table.put_item(Item=fishfry)